#### Tasks Exploration: 
   
>1. Setup GPU instance | jupyter env.   
>2. Try out using multiple GPUs with Keras on MNIST  
    i)  Check Keras.utils for multiple GPUs support  
    ii) Test with/out multiple GPUS

*h-rm_tan 15Dec2017*

In [ ]:
# !pwd
# /home/ubuntu/Expt

### Refs:  
KERAS API
- https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html 
- https://github.com/keras-team/keras/tree/master/keras/utils   

This uses mxnet
- https://github.com/NVIDIA/keras  

Some example code
- https://github.com/normanheckscher/mnist-multi-gpu/blob/master/mnist_multi_gpu_keras.py  
- https://github.com/kuza55/keras-extras/blob/master/examples/mnist_cnn_multi.py

Use TensorFlow Backend instead
- https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/

#### Helpful 1st test 
- https://keras.io/utils/#multi_gpu_model

#### To get multiple_gpu_model to load up
- https://stackoverflow.com/questions/46840037/cannot-import-multi-gpu-model-from-keras-utils

NEED TO INSTALL the latest version of Keras from github to get the multi_gpu_model module to load up on import

    sudo pip install git+git://github.com/fchollet/keras.git --upgrade
    sudo pip3 install git+git://github.com/fchollet/keras.git --upgrade

#### Test using multi-gpus with MNIST --- simple CNN model without much parameter tweaks
REF: https://github.com/normanheckscher/mnist-multi-gpu/blob/master/mnist_multi_gpu_keras.py

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

In [2]:
# https://machinelearningmastery.com/introduction-python-deep-learning-library-keras/
import os
os.environ["KERAS_BACKEND"]="tensorflow"

In [3]:
## Test load multi_gpu_model
from keras.utils import multi_gpu_model
# Using TensorFlow backend.

Using TensorFlow backend.


In [5]:
# load libraries
from tensorflow.contrib.keras.api.keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from keras import backend as K

from keras.callbacks import TensorBoard, ModelCheckpoint
tensorboard = TensorBoard(log_dir='/home/ubuntu/Expt/MNIST_train', histogram_freq=1,
                          write_graph=True, write_images=False, embeddings_freq=1)

In [6]:
import time

In [7]:
## Check re resources 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 193545795524082108
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11330676327
locality {
  bus_id: 1
}
incarnation: 18142107889108613369
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:17.0"
, name: "/gpu:1"
device_type: "GPU"
memory_limit: 11330676327
locality {
  bus_id: 1
}
incarnation: 14679588593817377482
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:00:18.0"
, name: "/gpu:2"
device_type: "GPU"
memory_limit: 11330676327
locality {
  bus_id: 1
}
incarnation: 13058470020451344541
physical_device_desc: "device: 2, name: Tesla K80, pci bus id: 0000:00:19.0"
, name: "/gpu:3"
device_type: "GPU"
memory_limit: 11328684032
locality {
  bus_id: 1
}
incarnation: 9302012676846680521
physical_device_desc: "device: 3, name: Tesla K80, pci bus id: 0000:00:1a.0"
, name: "/gpu:4"
device_type: "GPU"
memory_limit: 11328684032
locality {
  bus_id: 1
}
incarnation: 15485

In [15]:
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


# convert class vectors to binary class matrices
nb_classes = 10

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
model = Sequential()

model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='valid',
                        input_shape=input_shape))

model.add(Activation('relu'))
model.add(Convolution2D(256, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Convolution2D(128, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
# Instantiate the base model (or "template" model).
# We recommend doing this with under a CPU device scope,
# so that the model's weights are hosted on CPU memory.
# Otherwise they may end up hosted on a GPU, which would
# complicate weight sharing.
with tf.device('/cpu:0'):
    Omodel = model

In [22]:
## Test with 1 GPU --------------------------------

with tf.device('/gpu:0'):
    
    model.compile(loss='categorical_crossentropy',
                           optimizer='rmsprop',
                           metrics=['accuracy'])

    batch_size = 128
    nb_epoch = 12
    ngpus=1

    start_time = time.time()
    model.fit(X_train, Y_train, batch_size=batch_size*ngpus, epochs=nb_epoch,
                      verbose=1, validation_data=(X_test, Y_test)) #, callbacks=[tensorboard])

    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score -- loss:', score[0])
    print('Test metrics -- accuracy:', score[1])
    duration = time.time() - start_time
    print('Total Duration (%.3f sec)' % duration)

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 55s 913us/step - loss: 0.0606 - acc: 0.9838 - val_loss: 0.0922 - val_acc: 0.9803
Epoch 2/12
60000/60000 [==============================] - 52s 869us/step - loss: 0.0699 - acc: 0.9828 - val_loss: 0.0357 - val_acc: 0.9910
Epoch 3/12
60000/60000 [==============================] - 52s 864us/step - loss: 0.0772 - acc: 0.9817 - val_loss: 0.0942 - val_acc: 0.9814
Epoch 4/12
60000/60000 [==============================] - 52s 863us/step - loss: 0.0804 - acc: 0.9811 - val_loss: 0.0529 - val_acc: 0.9904
Epoch 5/12
60000/60000 [==============================] - 52s 861us/step - loss: 0.0814 - acc: 0.9813 - val_loss: 0.0420 - val_acc: 0.9900
Epoch 6/12
60000/60000 [==============================] - 52s 859us/step - loss: 0.0865 - acc: 0.9805 - val_loss: 0.0623 - val_acc: 0.9839
Epoch 7/12
60000/60000 [==============================] - 51s 858us/step - loss: 0.0874 - acc: 0.9806 - val_loss: 0.

In [17]:
## Test with MULTIPLE GPUs --------------------------------

# Replicates the model on N--->max8 GPUs.
# This assumes that your machine has 8 available GPUs.
# ngpus = 8


def run_model_parallelgpus(ngpus):
    print("Using %i GPUs" %ngpus)
    
    parallel_model = multi_gpu_model(Omodel, gpus=ngpus)
    parallel_model.compile(loss='categorical_crossentropy',
                           optimizer='rmsprop',
                           metrics=['accuracy'])

    batch_size = 128
    nb_epoch = 12


    start_time = time.time()
    parallel_model.fit(X_train, Y_train, batch_size=batch_size*ngpus, epochs=nb_epoch,
                      verbose=1, validation_data=(X_test, Y_test)) #, callbacks=[tensorboard])

    score = parallel_model.evaluate(X_test, Y_test, verbose=0)
    print('Test score -- loss:', score[0])
    print('Test metrics -- accuracy:', score[1])
    
    duration = time.time() - start_time
    print('Total Duration (%.3f sec)' % duration)

In [19]:
run_model_parallelgpus(ngpus=2)

Using 2 GPUs
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 29s 490us/step - loss: 0.0343 - acc: 0.9907 - val_loss: 0.0301 - val_acc: 0.9914
Epoch 2/12
60000/60000 [==============================] - 27s 442us/step - loss: 0.0329 - acc: 0.9908 - val_loss: 0.0314 - val_acc: 0.9917
Epoch 3/12
60000/60000 [==============================] - 26s 436us/step - loss: 0.0317 - acc: 0.9912 - val_loss: 0.0252 - val_acc: 0.9936
Epoch 4/12
60000/60000 [==============================] - 26s 436us/step - loss: 0.0342 - acc: 0.9908 - val_loss: 0.0401 - val_acc: 0.9882
Epoch 5/12
60000/60000 [==============================] - 26s 439us/step - loss: 0.0355 - acc: 0.9904 - val_loss: 0.0537 - val_acc: 0.9903
Epoch 6/12
60000/60000 [==============================] - 27s 445us/step - loss: 0.0359 - acc: 0.9906 - val_loss: 0.0294 - val_acc: 0.9924
Epoch 7/12
60000/60000 [==============================] - 26s 438us/step - loss: 0.0367 - acc: 0.9910 -

In [20]:
run_model_parallelgpus(ngpus=4)

Using 4 GPUs
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 17s 285us/step - loss: 0.0418 - acc: 0.9909 - val_loss: 0.0497 - val_acc: 0.9937
Epoch 2/12
60000/60000 [==============================] - 14s 240us/step - loss: 0.0357 - acc: 0.9922 - val_loss: 0.0505 - val_acc: 0.9927
Epoch 3/12
60000/60000 [==============================] - 14s 240us/step - loss: 0.0368 - acc: 0.9922 - val_loss: 0.0537 - val_acc: 0.9911
Epoch 4/12
60000/60000 [==============================] - 15s 244us/step - loss: 0.0337 - acc: 0.9926 - val_loss: 0.0395 - val_acc: 0.9945
Epoch 5/12
60000/60000 [==============================] - 15s 243us/step - loss: 0.0380 - acc: 0.9918 - val_loss: 0.0739 - val_acc: 0.9904
Epoch 6/12
60000/60000 [==============================] - 15s 243us/step - loss: 0.0311 - acc: 0.9935 - val_loss: 0.0453 - val_acc: 0.9914
Epoch 7/12
60000/60000 [==============================] - 15s 242us/step - loss: 0.0306 - acc: 0.9932 -

In [21]:
run_model_parallelgpus(ngpus=8)

Using 8 GPUs
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 11s 189us/step - loss: 0.0340 - acc: 0.9927 - val_loss: 0.0469 - val_acc: 0.9940
Epoch 2/12
60000/60000 [==============================] - 8s 141us/step - loss: 0.0272 - acc: 0.9940 - val_loss: 0.0358 - val_acc: 0.9940
Epoch 3/12
60000/60000 [==============================] - 8s 142us/step - loss: 0.0260 - acc: 0.9948 - val_loss: 0.0554 - val_acc: 0.9940
Epoch 4/12
60000/60000 [==============================] - 8s 141us/step - loss: 0.0235 - acc: 0.9950 - val_loss: 0.0397 - val_acc: 0.9901
Epoch 5/12
60000/60000 [==============================] - 8s 142us/step - loss: 0.0254 - acc: 0.9947 - val_loss: 0.0311 - val_acc: 0.9933
Epoch 6/12
60000/60000 [==============================] - 8s 141us/step - loss: 0.0216 - acc: 0.9951 - val_loss: 0.0586 - val_acc: 0.9942
Epoch 7/12
60000/60000 [==============================] - 9s 143us/step - loss: 0.0269 - acc: 0.9947 - val_l

### MNIST Test Summary

Using 1 GPU 
Test score -- loss: 0.0527517839638 |
Test metrics -- accuracy: 0.9907 |
Total Duration (626.454 sec)

Using 2 GPUs
Test score -- loss: 0.0482255089384 |
Test metrics -- accuracy: 0.9909 |
Total Duration (322.038 sec)

Using 4 GPUs
Test score -- loss: 0.0681361730697 |
Test metrics -- accuracy: 0.9908 |
Total Duration (179.739 sec)

Using 8 GPUs
Test score -- loss: 0.0483050177622 |
Test metrics -- accuracy: 0.9946 |
Total Duration (111.724 sec)

### BUGS

    swig/python detected a memory leak of type 'int64_t *', no destructor found.
> https://github.com/tensorflow/tensorflow/issues/13359